<a href="https://it-omscholing.nl/locaties/hogeschool-rotterdam/">
<div>
<a><img src='pics/banner.PNG'/></a>
</div>
<div>
<a href=''><img src='pics/miw.PNG'/></a>
<em>Author: Jeroen Boogaard</em>
</div>
</a>

<h1>Practicum Computer Vision</h1>

<h2>Imports<h2>

In [ ]:
from PIL import Image
import io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from sklearn.model_selection import train_test_split
import random
import requests
import tensorflow as tf
# import tensorflow.keras as keras
keras = tf.keras
import zipfile
import pandas as pd

**Gebruik een <i>environment variable</i> voor het instellen van het loglevel van TensorFlow**

In [ ]:
os.environ['TF_CPP_MIN_LEVEL'] = '2'

<h2>Data Collection</h2>

<h3>Data genereren</h3>

In [ ]:
riverDirectory = "pics/2750/River"
edgeFiles = list()
 
for filename in os.listdir(riverDirectory):
    imgFile = os.path.join(riverDirectory, filename)
    edgeFiles.append(imgFile)
    
highwayDirectory = "pics/2750/Highway"

for filename in os.listdir(highwayDirectory):
    imgFile = os.path.join(highwayDirectory, filename)
    edgeFiles.append(imgFile)

txtFiles = edgeFiles    

print(txtFiles[1])

<h3>Data filteren</h3>

In [ ]:
txtFiles = [x for x in txtFiles if ".jpg" in x]
print(txtFiles[1])

<h3>Data inlezen</h3>

In [ ]:
imageObjects = np.zeros([len(txtFiles), 64, 64, 3])

i = 0

for pic in txtFiles:
  imageObjects[i] = np.asarray(Image.open(txtFiles[i])).astype('uint8')/255
  i += 1
    
print(imageObjects[0])    

**Gebruik de bestandsnamen voor het verkrijgen van de labels**

In [ ]:
imageLabels = np.empty(len(txtFiles), dtype = 'S20')

i = 0

for label in txtFiles:
    txtFiles[i] = label.replace("\\", "/").split('/')[2]
    i += 1
    
print(txtFiles[1])    

In [ ]:
# change them to integers in alphabetical order
labelNames, labelNumbers = np.unique(txtFiles, return_inverse=True)
labelDict = dict(zip(np.unique(labelNumbers), labelNames))

# print(np.unique(labelNumbers), labelNames)

# print('labelNames: ', labelNames)
# print('labelNumbers: ', labelNumbers)

np.array(np.unique(labelNumbers, return_counts=True)).T

<h3>Data splitten</h3>

In [ ]:
trainSet, testSet, trainLabels, testLabels = train_test_split(imageObjects, labelNumbers, stratify = labelNumbers, train_size = 0.75, random_state=42)
print(trainSet.shape)
nrOfImages = len(trainSet)

<h3>Data inspecteren</h3>

In [ ]:
def inspectData():
    
    plt.figure(figsize=(10,10))
    
    for i in range(16):
        plt.subplot(4, 4, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        number = i
        plt.imshow(trainSet[number])
        plt.xlabel(labelNames[trainLabels[number]]) 
    
    plt.show()

In [ ]:
inspectData()    

<h2>Maak een model</h2>

<h3>Definieer de netwerktopologie</h3>

<h3>Exercise 1</h3>
<p>
    Vul de netwerktopologie tot een bruikbaar Neural Netwerk onstaat.
</p>

In [ ]:
# batch_size = 64
img_height = 64
img_width = 64

In [ ]:
numberNames = len(labelNames)
# print(numberNames)

In [ ]:
model = tf.keras.Sequential([
  keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(64, 64, 3)),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(10)
])

<h3>Definieer de trainparameters</h3>

In [ ]:
lossFunction = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gdAlgorithm = tf.keras.optimizers.Adam(learning_rate=0.001)
nrOfEpochs = 10

<h3>Train het model</h3>

In [ ]:
model.compile(optimizer=gdAlgorithm, loss=lossFunction, metrics="accuracy")

In [ ]:
model.summary()

In [ ]:
history = model.fit(trainSet, trainLabels, epochs=nrOfEpochs, batch_size=64, verbose=2, validation_data=(testSet, testLabels))

In [ ]:
model.save("models_saved/model_2.h5", overwrite=True, include_optimizer=True)

In [ ]:
model.predict(testSet)

In [ ]:
# np.argmax om: Classification matrix can't handle a mix of binary and continous multi-output targets

predictionBatch = model.predict(testSet)
predictionEnhanced = np.argmax(predictionBatch, axis=-1)

print(predictionEnhanced)

# original labels

labels = np.argmax(testLabels,axis=-1)

# from sklearn.metrics import classification_report

# print(classification_report(labels, pred))

conf_matrix = tf.math.confusion_matrix(labels=testLabels, predictions=predictionEnhanced)

print(conf_matrix)

# <h2>**Test het model**</h2>

<h3>Exercise 2</h3>
<p>
    Test het Neural Netwerk met de testset en laat zien of het in staat is de juiste class te vinden.
</p>

In [ ]:
from sklearn import metrics

actual = testLabels
predicted = predictionEnhanced

confusion_matrix = metrics.confusion_matrix(actual, predicted)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["Highway", "River"])

# Origineel: display_labels = [False, True]

cm_display.plot()
plt.show()

In [ ]:
predictions = predictionEnhanced

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(testLabels,predictions))
print(classification_report(testLabels,predictions))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = nrOfEpochs
epochs_range = range(epochs)

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='upper left')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
landscape_url = "http://www.freeimageslive.co.uk/files/images010/fields_pathwork.preview.jpg"
# landscape_url = "https://wallup.net/wp-content/uploads/2016/01/248765-Earth-landscape-aerial_view-nature.jpg"
# landscape_url = 'https://drive.google.com/file/d/1YuQ-hShFq8-GZuqT1LrCAil_2ogs9gl9/view?usp=sharing'
# landscape_url = "https://www.denarend.com/satellite-sculpture-views/images/dirksland-bridge-02.jpg"
# landscape_url = "https://i0.wp.com/lifeology101.com/wp-content/uploads/2017/08/img_6015-copy.jpg?resize=665%2C443&ssl=1"
landscape_path = tf.keras.utils.get_file('agricultural_landscape', origin=landscape_url)

# landscape_img = "https://www.denarend.com/satellite-sculpture-views/images/dirksland-bridge-02.jpg"
# landscape_url = "https://i0.wp.com/lifeology101.com/wp-content/uploads/2017/08/img_6015-copy.jpg?resize=665%2C443&ssl=1"
# landscape_path = tf.keras.utils.get_file('agricultural_landscape', origin=landscape_img)


print(landscape_url)

img = tf.keras.utils.load_img(
    landscape_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

# img.show()

predictions = predictionEnhanced
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(labelNames[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
# #hier kies je welke index je iets mee wilt
# i = 1000


# #dit is het plaatje van index i
# img = testSet[i]


# # print de label naam van index i
# print(labelNames[testLabels[i]])

# for i in range(16):

# #laat plaatje zien
# plt.imshow(img)

# predictions = model.predict(img)
# score = tf.nn.softmax(predictions[0])

# print(
#     "This image most likely belongs to {} with a {:.2f} percent confidence."
#     .format(labelNames[np.argmax(score)], 100 * np.max(score))
# )